In [1]:
from Preprocessor import Preprocessor
from OfflineEvaluator import OfflineEvaluator
from ContentAnalyzer import ContentAnalyzer
from PostFilter import PostFilter
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import itertools
import pickle
from STOPWORDS import STOP_WORDS
import seaborn as sns
from scipy import stats
from collections import Counter
from itertools import combinations
import utils
from datetime import datetime
import networkx as nx

In [2]:
# Sample course SA
dbnum = ##
num_weeks = 6
START_DATE = #datetime(...)
TEMPORAL_START_WEEK = 3
instructors = #[...]

In [4]:
k = 10
week_stride = 1
path_clean = "cleaned_data"
path_sample = "sample"

# preprocess
WEIGHT_POST_LOOKUP = {1: 'created',2:'liked',3:'linked',4:'replied',5:'revisited',6:'read',7:'anonymously read'}
WEIGHT_USER_LOOKUP = {1:'liked',2:'linked',3:'replied',4:'read',5:'anonymously read'}
CUTOFF_WEIGHT = 5
EXPLICIT_THRESHOLD = 5
IMPLICIT_THRESHOLD = 10

In [5]:
preprocessor = Preprocessor(path_sample, dbnum, 
                            WEIGHT_POST_LOOKUP, WEIGHT_USER_LOOKUP, START_DATE, verbose=0)

/home/zhaorui/anaconda3/envs/main/lib/python3.7/site-packages/pandas/core/indexing.py:1418: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


In [6]:
# preprocessor.preview_interaction_distribution()

# Universal Sentence Encoder embeddings and similarities

In [8]:
# Item similarities
precomputed_item_similarities = {}
feature_size = 128 # for USE

# IF USE TFIDF
#precomputed_item_similarities = self.preprocessor.precompute_similarity('', 'tfidf+lsi', 64)

# if use USE
precomputed_item_similarities, sentence_embs = preprocessor.precompute_similarity('lib/use_pretrained_models/1', 
                                                                                  'sentence_emb', feature_size)
# if use BERT
#precomputed_item_similarities = self.preprocessor.precompute_similarity('cleaned_data/ft_model_incr', 'bert_word_emd+sif', 300)

In [9]:
with open('cleaned_data/db%d/precomputed_item_similarities_use3.pickle'%dbnum, 'wb') as handle:
    pickle.dump(precomputed_item_similarities, handle)
with open('cleaned_data/db%d/precomputed_sen_embs.pickle'%dbnum, 'wb') as handle:
    pickle.dump(sentence_embs, handle)

# Content graph

In [11]:
evaluator = OfflineEvaluator(WEIGHT_POST_LOOKUP, WEIGHT_USER_LOOKUP, CUTOFF_WEIGHT, 
                             EXPLICIT_THRESHOLD, IMPLICIT_THRESHOLD, START_DATE, verbose=0)
evaluator.set_ground_truth_criterion('explicit_implicit')
evaluator.preprocess(path_sample, dbnum, num_weeks, week_stride)

/home/zhaorui/anaconda3/envs/main/lib/python3.7/site-packages/pandas/core/indexing.py:1418: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


In [ ]:
keyword_nums = 10
ca = ContentAnalyzer(evaluator.all_note_contents)
ca.preprocess(['NOUN','VERB','PROPN'])
ca.extract_keywords(keyword_nums)
graph = ca.construct_hypernym_graph()
#print(nx.info(graph))
nodes_hyp = [x for x,y in graph.nodes(data=True) if y['node_type']=='hypernym']
nodes_hyp

In [13]:
nx.write_graphml_lxml(graph, "cleaned_data/db%d/content_graph-%d.graphml"%(dbnum,keyword_nums))

In [ ]:
keyword_nums = 5
ca = ContentAnalyzer(evaluator.all_note_contents)
ca.preprocess(['NOUN','VERB','PROPN'])
ca.extract_keywords(keyword_nums)
graph = ca.construct_hypernym_graph()
#print(nx.info(graph))
nodes_hyp = [x for x,y in graph.nodes(data=True) if y['node_type']=='hypernym']
nodes_hyp

In [15]:
nx.write_graphml_lxml(graph, "cleaned_data/db%d/content_graph-%d.graphml"%(dbnum,keyword_nums))

In [ ]:
keyword_nums = 0.2
ca = ContentAnalyzer(evaluator.all_note_contents)
ca.preprocess(['NOUN','VERB','PROPN'])
ca.extract_keywords(keyword_nums)
graph = ca.construct_hypernym_graph()
#print(nx.info(graph))
nodes_hyp = [x for x,y in graph.nodes(data=True) if y['node_type']=='hypernym']
nodes_hyp

In [17]:
nx.write_graphml_lxml(graph, "cleaned_data/db%d/content_graph-%.1f.graphml"%(dbnum,keyword_nums))

# Post filter and Post Creation Dates

In [18]:
'''
Calculate post creation dates
'''

lst = []
post_create_dates_all = {}
ct = 0
for u, df_temp in evaluator.all_post_inters.items(): #NOTE only use trian post inter only
    #print(u)
    df_temp = df_temp.loc[df_temp.weight==1,['NoteID','Weeknum']]    
    lst.append(df_temp.set_index('NoteID').to_dict()['Weeknum'])
    ct+=len(df_temp)
for dic in lst:
    for k,v in dic.items():
        post_create_dates_all[k] = v

In [ ]:
pfilter = PostFilter()
pfilter.fit(evaluator.all_note_contents)
pfilter.prune_on_length(3)
pfilter.prune_on_awl(3)

In [20]:
with open('cleaned_data/db%d/pfilter.pickle'%dbnum, 'wb') as handle:
    pickle.dump(pfilter, handle)
with open('cleaned_data/db%d/post_create_dates_all.pickle'%dbnum, 'wb') as handle:
    pickle.dump(post_create_dates_all, handle)

# Keywords

In [21]:
def get_keywords(all_note_contents):
    data_words, note_ids, id2word, corpus = utils.extract_vocab(all_note_contents, 
                                                                5, ['NOUN','VERB'], STOP_WORDS)
    tfidf_matrix, tf_dicts, post_appear_dict = utils.tfidf(data_words)
    keywords = {i: utils.get_top_tfidfs(tfidf_matrix[i], len(tfidf_matrix[i]) // 5) 
                for i, m in enumerate(tfidf_matrix)}
    keywords
    return note_ids, keywords

In [22]:
note_ids, keywords = get_keywords(preprocessor.all_note_contents)

In [23]:
keywords = {note_ids[i]:m for i, m in keywords.items()}

In [24]:
with open('cleaned_data/db%d/precomputed_keyword_lookups.pickle'%dbnum, 'wb') as handle:
    pickle.dump(keywords, handle)